<a href="https://colab.research.google.com/github/cindyangelira/cindyangelira/blob/main/Kraken_Firebase_uuid_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup Bigquery

from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

In [ ]:
# @title Dependencies

import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime

In [ ]:
project1 = 'fullstackcustomer' # Project ID inserted based on the query results selected to explore
location1 = 'asia-southeast1' # Location inserted based on the query results selected to explore
client1 = bigquery.Client(project=project1, location=location1)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
# @title Query Firebase Analytic

sql_query1 = ('''SELECT 
event_date,event_timestamp, user_pseudo_id, 
event_name, (SELECT value.string_value FROM UNNEST(event_params)
WHERE key = "user_uuid") AS uuid
FROM `fullstackcustomer.analytics_212122878.events_*`''')

In [ ]:
# @title Store to Dataframe

firebase_df = client1.query(sql_query1).to_dataframe()
firebase_df.head()

,event_date,event_timestamp,user_pseudo_id,event_name,uuid
0,20221112,1668217641160000,89c5ad85fc48763ee53af1da4285c497,app_update,None
1,20221112,1668232893803000,9e82eb91dba05df10824141bfb4dd692,app_update,None
2,20221112,1668269022931000,8e66ff7fad01d3f5028de69bc8ea5133,app_update,None
3,20221112,1668222551274000,6766c5b658ec2135766e25c0668a1182,app_update,None
4,20221112,1668261176667000,14210b4ed1d3b3c78dc780a584e8cee9,app_update,None


In [ ]:
firebase_df['event_date'] = pd.to_datetime(firebase_df.event_date)
firebase_df['event_timestamp'] = firebase_df['event_timestamp']/(10**6)
firebase_df['event_timestamp'] = [datetime.fromtimestamp(x) for x in firebase_df['event_timestamp']]
firebase_df.head()

,event_date,event_timestamp,user_pseudo_id,event_name,uuid
0,2022-11-12,2022-11-12 01:47:21.160,89c5ad85fc48763ee53af1da4285c497,app_update,None
1,2022-11-12,2022-11-12 06:01:33.803,9e82eb91dba05df10824141bfb4dd692,app_update,None
2,2022-11-12,2022-11-12 16:03:42.931,8e66ff7fad01d3f5028de69bc8ea5133,app_update,None
3,2022-11-12,2022-11-12 03:09:11.274,6766c5b658ec2135766e25c0668a1182,app_update,None
4,2022-11-12,2022-11-12 13:52:56.667,14210b4ed1d3b3c78dc780a584e8cee9,app_update,None


In [ ]:
min(firebase_df.event_timestamp), max(firebase_df.event_timestamp)

(Timestamp('2022-09-22 00:00:00.222000'),
 Timestamp('2022-11-20 23:57:47.407014'))

In [ ]:
# @title Query Kraken Customers Orders

project2 = 'octopus-data-prod-294206'
location2 = 'US'
client2 = bigquery.Client(project = project2, location=location2)

sql_query2 = ('''
WITH users AS(
  SELECT *,
  CASE WHEN premium_id IS NULL THEN 'not premium' ELSE 'premium' END AS premium_status
  FROM `octopus-data-prod-294206.KRAKEN_temp_view.customer_users`
  ),
orders AS(
  SELECT * FROM `octopus-data-prod-294206.KRAKEN_temp_view.customer_orders`
  )
SELECT t1.*, t2.uuid, t2.premium_status
FROM orders t1
LEFT JOIN users t2 ON t1.user_id = t2.user_id
''')


In [ ]:
kraken_order = client2.query(sql_query2).to_dataframe()
kraken_order.head()

,order_id,system,timestamp,total_quantity,total_amount,total_weight,user_id,receiver_id,region_id,region_name,...,country,province,city,district,village,order_status,route_id,total_distance,uuid,premium_status
0,74a2fdf7-6c62-4b2b-8402-bd729feef14b,V2,2021-06-08 04:28:43.991329,10.000000000,0E-9,250.000000000,66711,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Bandung,Kecamatan Cangkuang,Ciluncat,cancelled,<NA>,NaN,cb92c691-c767-4ec6-a669-a9def607da8a,not premium
1,c0a1de17-591e-4d34-ab05-38a4255b23f1,V2,2021-06-14 03:48:58.558019,12.000000000,0E-9,250.000000000,64836,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Bandung,Cileunyi,Cinunuk,cancelled,<NA>,NaN,19074c45-fbae-4597-b65b-8efd15218fb0,premium
2,95f46fde-9b93-4069-877d-09a185cfdfb9,V2,2021-06-13 03:45:30.140950,12.000000000,0E-9,250.000000000,64836,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Bandung,Cileunyi,Cinunuk,cancelled,<NA>,NaN,19074c45-fbae-4597-b65b-8efd15218fb0,premium
3,fa60b03a-cfde-4eac-81da-a98aaa0b4f3c,V2,2021-06-27 08:19:03.694438,28.000000000,0E-9,250.000000000,67965,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Bandung,Cileunyi,Cimekar,cancelled,<NA>,NaN,a0006b8c-0a33-4ee0-b616-c38282362b6c,not premium
4,3dabbe2f-0241-43b3-a53e-b78bdabe963f,V2,2021-07-22 05:05:18.865012,11.000000000,6.000000000,313.510000000,67197,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Bandung,Cileunyi,Cileunyi Kulon,cancelled,<NA>,NaN,cc80ca92-be79-476c-8cb1-2c2b8f68c43e,not premium


In [ ]:
kraken_order.shape

(46043, 22)

In [ ]:
kraken_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46043 entries, 0 to 46042
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        46043 non-null  object        
 1   system          46043 non-null  object        
 2   timestamp       46043 non-null  datetime64[ns]
 3   total_quantity  41139 non-null  object        
 4   total_amount    41139 non-null  object        
 5   total_weight    41139 non-null  object        
 6   user_id         46043 non-null  Int64         
 7   receiver_id     31369 non-null  Int64         
 8   region_id       46043 non-null  Int64         
 9   region_name     46043 non-null  object        
 10  latitude        46043 non-null  float64       
 11  longitude       46043 non-null  float64       
 12  country         45856 non-null  object        
 13  province        45830 non-null  object        
 14  city            45830 non-null  object        
 15  di

In [ ]:
# @title Filter Kraken Based on Firebase Date

kraken_order_filter = kraken_order[(kraken_order['timestamp'] >= min(firebase_df.event_timestamp)) & (kraken_order['timestamp'] <= max(firebase_df.event_timestamp))]

In [ ]:
kraken_order_filter.head()

,order_id,system,timestamp,total_quantity,total_amount,total_weight,user_id,receiver_id,region_id,region_name,...,country,province,city,district,village,order_status,route_id,total_distance,uuid,premium_status
73,26425f62-9d32-4ff3-9aee-28c6ba1935bc,V2,2022-11-12 05:08:59.876230,10.000000000,20.000000000,68.411000000,117146,<NA>,2,Bali,...,Indonesia,Bali,Kabupaten Klungkung,Kecamatan Klungkung,Semarapura Kelod,cancelled,<NA>,NaN,f15c301a-7948-4b80-a8e1-50e8b4203241,not premium
97,c185af39-30eb-40ba-993a-5735489318de,V2,2022-10-04 06:36:03.297683,None,None,None,111879,1,3,Other Indonesia,...,Indonesia,Jawa Barat,Kota Depok,Kecamatan Sawangan,Sawangan Baru,cancelled,<NA>,NaN,61bd4559-b75b-461d-83a9-08476490b981,not premium
101,d06d9816-1226-48e6-bc06-a7a410e1e513,V2,2022-11-14 01:33:50.106634,16.000000000,0E-9,5.000000000,109482,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Kota Bandung,Bandung Kulon,Cigondewah Rahayu,cancelled,<NA>,NaN,1be8a4c5-d4e8-4705-b325-3c25d192d561,not premium
108,a4a447ad-3c55-415e-b81a-d1cf01590bde,V2,2022-11-02 04:43:17.953163,10.000000000,13.000000000,37.271000000,81422,<NA>,8,Bandung,...,Indonesia,Jawa Barat,Kabupaten Bandung,Kecamatan Rancaekek,Bojongloa,cancelled,<NA>,NaN,1ed87bf7-dbd9-4a8f-aa90-cc634f5c6c5c,premium
110,b6aac303-f0d1-4d06-9a44-8d375ff03f1d,V2,2022-11-19 18:01:55.537190,10.000000000,10.000000000,15.151000000,117979,<NA>,2,Bali,...,Indonesia,Bali,Denpasar,Denpasar Utara,Ubung,cancelled,<NA>,NaN,53991a5d-57ba-4e5f-bcd6-f6001b88cc13,not premium


In [ ]:
min(kraken_order_filter.timestamp), max(kraken_order_filter.timestamp)

(Timestamp('2022-09-22 01:36:18.684898'),
 Timestamp('2022-11-20 23:04:42.412080'))

In [ ]:
# @title Extract uuid from Kraken and Firebase

uuid_kraken = kraken_order_filter.dropna(subset=['uuid']).uuid.unique()
uuid_firebase = firebase_df.dropna(subset = ['uuid']).uuid.unique()

In [ ]:
len(uuid_kraken), len(uuid_firebase)

(3614, 11975)

In [ ]:
# @title Add New Column *user_in_firebase* on Kraken Customer Orders
# yes if user uuid in Kraken is also captured in firebase, otherwise no

kraken_order_filter['user_in_firebase'] = np.where(kraken_order_filter['uuid'].isin(uuid_firebase), "Yes", "No")
kraken_order_filter.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,order_id,system,timestamp,total_quantity,total_amount,total_weight,user_id,receiver_id,region_id,region_name,...,province,city,district,village,order_status,route_id,total_distance,uuid,premium_status,user_in_firebase
73,26425f62-9d32-4ff3-9aee-28c6ba1935bc,V2,2022-11-12 05:08:59.876230,10.000000000,20.000000000,68.411000000,117146,<NA>,2,Bali,...,Bali,Kabupaten Klungkung,Kecamatan Klungkung,Semarapura Kelod,cancelled,<NA>,NaN,f15c301a-7948-4b80-a8e1-50e8b4203241,not premium,Yes
97,c185af39-30eb-40ba-993a-5735489318de,V2,2022-10-04 06:36:03.297683,None,None,None,111879,1,3,Other Indonesia,...,Jawa Barat,Kota Depok,Kecamatan Sawangan,Sawangan Baru,cancelled,<NA>,NaN,61bd4559-b75b-461d-83a9-08476490b981,not premium,No
101,d06d9816-1226-48e6-bc06-a7a410e1e513,V2,2022-11-14 01:33:50.106634,16.000000000,0E-9,5.000000000,109482,<NA>,8,Bandung,...,Jawa Barat,Kota Bandung,Bandung Kulon,Cigondewah Rahayu,cancelled,<NA>,NaN,1be8a4c5-d4e8-4705-b325-3c25d192d561,not premium,Yes
108,a4a447ad-3c55-415e-b81a-d1cf01590bde,V2,2022-11-02 04:43:17.953163,10.000000000,13.000000000,37.271000000,81422,<NA>,8,Bandung,...,Jawa Barat,Kabupaten Bandung,Kecamatan Rancaekek,Bojongloa,cancelled,<NA>,NaN,1ed87bf7-dbd9-4a8f-aa90-cc634f5c6c5c,premium,Yes
110,b6aac303-f0d1-4d06-9a44-8d375ff03f1d,V2,2022-11-19 18:01:55.537190,10.000000000,10.000000000,15.151000000,117979,<NA>,2,Bali,...,Bali,Denpasar,Denpasar Utara,Ubung,cancelled,<NA>,NaN,53991a5d-57ba-4e5f-bcd6-f6001b88cc13,not premium,Yes


In [ ]:
kraken_order_filter.user_in_firebase.value_counts()

Yes    7631
No      889
Name: user_in_firebase, dtype: int64

In [ ]:
# @title Print Kraken Orders Data that *uuid* is not Captured in Firebase

kraken_order_filter[kraken_order_filter['user_in_firebase']=='No'].head()

,order_id,system,timestamp,total_quantity,total_amount,total_weight,user_id,receiver_id,region_id,region_name,...,province,city,district,village,order_status,route_id,total_distance,uuid,premium_status,user_in_firebase
97,c185af39-30eb-40ba-993a-5735489318de,V2,2022-10-04 06:36:03.297683,None,None,None,111879,1,3,Other Indonesia,...,Jawa Barat,Kota Depok,Kecamatan Sawangan,Sawangan Baru,cancelled,<NA>,NaN,61bd4559-b75b-461d-83a9-08476490b981,not premium,No
195,8256d540-dbc9-48bb-8100-184fa3933cc8,V2,2022-10-24 18:56:52.331662,2.000000000,190000.000000000,13000.000000000,113677,<NA>,13,Jakarta,...,Banten,Kota Tangerang Selatan,Kecamatan Pamulang,Benda Baru,cancelled,<NA>,NaN,4e9422dc-646d-4ee9-9116-4a7368a72551,not premium,No
209,457de831-b865-4395-8605-e9879b6fe7a4,V2,2022-10-24 02:01:58.630472,1.000000000,40000.000000000,3000.000000000,113618,<NA>,13,Jakarta,...,Banten,Kota Tangerang Selatan,Kecamatan Serpong Utara,Pakualam,cancelled,<NA>,NaN,08bf890d-daa3-44d4-a43b-2f1fe1bbbe3f,not premium,No
217,9d14e268-63de-4890-a454-dcd734573379,V2,2022-11-18 05:42:46.741114,3.000000000,800.000000000,1000.000000000,109059,1706,13,Jakarta,...,Jawa Barat,Kota Bekasi,Pondokmelati,Jatimelati,cancelled,<NA>,NaN,08a84791-be60-4912-b016-2234943ba63a,not premium,No
225,800df737-f5b4-4160-af1b-ca3e8a286396,V2,2022-10-27 15:23:07.827094,5.500000000,16650.000000000,4000.000000000,113068,<NA>,13,Jakarta,...,Daerah Khusus Ibukota Jakarta,Kota Jakarta Barat,Kecamatan Cengkareng,Duri Kosambi,cancelled,<NA>,NaN,0c602f8d-c92a-4309-b160-172c849df939,not premium,No


In [ ]:
kraken_order_filter[kraken_order_filter['user_in_firebase']=='No'].uuid.nunique()

620

In [ ]:
# @title Add New Column *user_in_kraken* on Firebase Analytic

# yes if user uuid in Kraken is also captured in firebase, otherwise no

firebase_df['user_in_kraken'] = np.where(firebase_df['uuid'].isin(uuid_kraken), "Yes", "No")
firebase_df.head()

,order_id,system,timestamp,total_quantity,total_amount,total_weight,user_id,receiver_id,region_id,region_name,...,province,city,district,village,order_status,route_id,total_distance,uuid,premium_status,user_in_firebase
73,26425f62-9d32-4ff3-9aee-28c6ba1935bc,V2,2022-11-12 05:08:59.876230,10.000000000,20.000000000,68.411000000,117146,<NA>,2,Bali,...,Bali,Kabupaten Klungkung,Kecamatan Klungkung,Semarapura Kelod,cancelled,<NA>,NaN,f15c301a-7948-4b80-a8e1-50e8b4203241,not premium,Yes
97,c185af39-30eb-40ba-993a-5735489318de,V2,2022-10-04 06:36:03.297683,None,None,None,111879,1,3,Other Indonesia,...,Jawa Barat,Kota Depok,Kecamatan Sawangan,Sawangan Baru,cancelled,<NA>,NaN,61bd4559-b75b-461d-83a9-08476490b981,not premium,No
101,d06d9816-1226-48e6-bc06-a7a410e1e513,V2,2022-11-14 01:33:50.106634,16.000000000,0E-9,5.000000000,109482,<NA>,8,Bandung,...,Jawa Barat,Kota Bandung,Bandung Kulon,Cigondewah Rahayu,cancelled,<NA>,NaN,1be8a4c5-d4e8-4705-b325-3c25d192d561,not premium,Yes
108,a4a447ad-3c55-415e-b81a-d1cf01590bde,V2,2022-11-02 04:43:17.953163,10.000000000,13.000000000,37.271000000,81422,<NA>,8,Bandung,...,Jawa Barat,Kabupaten Bandung,Kecamatan Rancaekek,Bojongloa,cancelled,<NA>,NaN,1ed87bf7-dbd9-4a8f-aa90-cc634f5c6c5c,premium,Yes
110,b6aac303-f0d1-4d06-9a44-8d375ff03f1d,V2,2022-11-19 18:01:55.537190,10.000000000,10.000000000,15.151000000,117979,<NA>,2,Bali,...,Bali,Denpasar,Denpasar Utara,Ubung,cancelled,<NA>,NaN,53991a5d-57ba-4e5f-bcd6-f6001b88cc13,not premium,Yes


In [ ]:
firebase_df.iloc[:,-1].value_counts()

No     1031540
Yes     599882
Name: user_in_kraken, dtype: int64

In [ ]:
def plot_count(col, labels): #labels only take string
  count_table = pd.DataFrame(col.value_counts())
  fig = px.bar(count_table, x=count_table.iloc[:,0], color = count_table.index, labels= {'x':'Count', 'index':''}, title = 'How many transaction that also captured in '+labels+' ?')
  fig.show()

In [ ]:
# @title Count Transaction in Kraken that Captured in Firebase

plot_count(kraken_order_filter.user_in_firebase, labels = 'Firebase')

In [ ]:
# @title Count Transaction in Firebase that Captured in Kraken

plot_count(firebase_df.user_in_kraken, labels = 'Kraken')

In [ ]:
# @title Let's Assume That Our Main Focus is Number of User That Not Captured

kraken_order_filter_user = kraken_order_filter.drop_duplicates(subset = ['uuid'])
firebase_user_df = firebase_df.drop_duplicates(subset = ['uuid'])

In [ ]:
# @title Count *uuid* in Kraken that Captured in Firebase

plot_count(kraken_order_filter_user.user_in_firebase, labels = 'Firebase')

In [ ]:
# @title Count *uuid* in Firebase that Captured in Kraken

plot_count(firebase_user_df.user_in_kraken, labels = 'Kraken')